In [1]:
# Import necessary libraries & paths 
import os
import json
import joblib
import numpy as np
import pandas as pd

MODELS_DIR = "../models"  # this notebook lives in notebooks/, so ../models is correct

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load trained ML assets (crop)

# Crop model expects features in order: [N, P, K, temperature, humidity, pH, rainfall]
crop_model_path = os.path.join(MODELS_DIR, "crop_recommender.pkl")
crop_scaler_path = os.path.join(MODELS_DIR, "crop_scaler.pkl")

crop_model = joblib.load(crop_model_path)
crop_scaler = joblib.load(crop_scaler_path)

print("✅ Loaded:", crop_model_path)
print("✅ Loaded:", crop_scaler_path)


✅ Loaded: ../models\crop_recommender.pkl
✅ Loaded: ../models\crop_scaler.pkl


In [4]:
# Define soil types from your dataset (fixed 4 classes) 
SOIL_TYPES = [
    "Red and Yellow soils",
    "Alluvial soils",
    "Laterite soils",
    "Black soils",
]

print("Soil types (from dataset):", SOIL_TYPES)

Soil types (from dataset): ['Red and Yellow soils', 'Alluvial soils', 'Laterite soils', 'Black soils']


In [5]:
# Rule tables (covers ALL 22 crops from your trained crop model) 

# Base crop water requirement (approx) in mm/week (typical range; tuned to be practical)
# These are general-purpose values to start; you can refine per region/growth stage later.
CROP_BASE_MM_PER_WEEK = {
    # Fruits
    "apple":        35,
    "banana":       55,
    "grapes":       40,
    "mango":        45,
    "orange":       40,
    "papaya":       55,
    "pomegranate":  45,
    "muskmelon":    35,
    "watermelon":   40,

    # Plantation / cash crops
    "coconut":      60,
    "coffee":       55,
    "cotton":       45,
    "jute":         50,

    # Cereals
    "rice":         65,
    "maize":        45,

    # Pulses / legumes
    "blackgram":    30,
    "chickpea":     30,
    "kidneybeans":  32,
    "lentil":       28,
    "mothbeans":    26,
    "mungbean":     26,
    "pigeonpeas":   35,
}

# Soil multipliers for your 4 soil classes
# Laterite: porous (higher need). Red & Yellow: lighter (slightly higher).
# Alluvial: balanced. Black: clayey, high WHC (lower need).
SOIL_MULTIPLIER = {
    "Red and Yellow soils": 1.10,
    "Alluvial soils":       1.00,
    "Laterite soils":       1.15,
    "Black soils":          0.85,
}

# Weather adjustments:
# - Temperature >30°C increases need ~3% per °C above 30
# - Humidity >60% reduces need ~1% per % above 60 (capped)
TEMP_INCREASE_PER_C_ABOVE_30 = 0.03
HUMIDITY_REDUCTION_PER_PC_ABOVE_60 = 0.01
MAX_HUMIDITY_REDUCTION = 0.25  # cap at -25%


In [6]:
# Rule-based irrigation calculator 

def calculate_irrigation_mm_per_week(
    crop_label: str,
    soil_type: str,
    temperature_c: float,
    humidity_pc: float,
    rainfall_mm_week: float
) -> dict:
    """
    Returns a dict with:
      - crop (lowercase string)
      - soil_type
      - base_mm_week
      - temp_factor
      - humidity_factor
      - soil_factor
      - effective_rain_mm
      - irrigation_mm_week
      - category (Low/Medium/High/Very High)
      - notes
    """
    crop = str(crop_label).lower()
    base = CROP_BASE_MM_PER_WEEK.get(crop, 40)  # default baseline if unseen

    # Soil factor
    soil_factor = SOIL_MULTIPLIER.get(soil_type, 1.0)

    # Temperature factor
    if temperature_c > 30:
        temp_factor = 1.0 + (temperature_c - 30.0) * TEMP_INCREASE_PER_C_ABOVE_30
    else:
        temp_factor = 1.0  # you can reduce below 30 if you like; keeping simple

    # Humidity factor (reduce water if humidity is high; cap reduction)
    if humidity_pc > 60:
        reduction = min((humidity_pc - 60.0) * HUMIDITY_REDUCTION_PER_PC_ABOVE_60, MAX_HUMIDITY_REDUCTION)
        humidity_factor = 1.0 - reduction
    else:
        humidity_factor = 1.0

    # Combine factors
    gross_need = base * soil_factor * temp_factor * humidity_factor

    # Effective rainfall: assume 80% effective up to gross need (simple agronomic rule)
    effective_rain = min(rainfall_mm_week * 0.8, gross_need)

    irrigation = max(gross_need - effective_rain, 0.0)

    # Categorize irrigation requirement for ease of use
    if irrigation < 20:
        cat = "Low"
    elif irrigation < 40:
        cat = "Medium"
    elif irrigation < 60:
        cat = "High"
    else:
        cat = "Very High"

    notes = []
    if soil_type == "Laterite soils":
        notes.append("Porous soil – consider more frequent, lighter irrigations.")
    if soil_type == "Black soils":
        notes.append("Cracking clay – use deeper but less frequent irrigations.")
    if temperature_c > 35:
        notes.append("High evapotranspiration risk (>35°C). Monitor more often.")
    if humidity_pc > 80:
        notes.append("High humidity – reduce irrigation slightly to avoid disease.")

    return {
        "crop": crop,
        "soil_type": soil_type,
        "base_mm_week": round(base, 2),
        "soil_factor": round(soil_factor, 2),
        "temp_factor": round(temp_factor, 2),
        "humidity_factor": round(humidity_factor, 2),
        "effective_rain_mm": round(effective_rain, 2),
        "irrigation_mm_week": round(irrigation, 2),
        "category": cat,
        "notes": notes
    }


In [7]:
# Helper - predict crop from N,P,K,Temp,Humidity,pH,Rainfall

def predict_crop_from_features(N, P, K, temperature, humidity, ph, rainfall):
    arr = np.array([[N, P, K, temperature, humidity, ph, rainfall]], dtype=float)
    arr_scaled = crop_scaler.transform(arr)
    pred = crop_model.predict(arr_scaled)[0]
    return str(pred)  # ensure string for downstream usage


In [8]:
# End-to-end pipeline function ---
# You provide soil type (from your 4 classes) and raw agronomic inputs.
# Pipeline predicts the crop, then computes irrigation need (mm/week) using rule-based logic.

def irrigation_pipeline(
    soil_type: str,
    N: float,
    P: float,
    K: float,
    temperature: float,
    humidity: float,
    ph: float,
    rainfall_mm_week: float
) -> dict:
    # Validate soil type
    if soil_type not in SOIL_TYPES:
        raise ValueError(f"Soil type '{soil_type}' not in dataset classes: {SOIL_TYPES}")

    # 1) Predict crop from your trained model
    predicted_crop = predict_crop_from_features(N, P, K, temperature, humidity, ph, rainfall_mm_week)

    # 2) Rule-based irrigation calculation
    irri = calculate_irrigation_mm_per_week(
        crop_label=predicted_crop,
        soil_type=soil_type,
        temperature_c=temperature,
        humidity_pc=humidity,
        rainfall_mm_week=rainfall_mm_week
    )

    # 3) Bundle response
    return {
        "Predicted Crop": predicted_crop,
        "Soil Type": soil_type,
        "Irrigation (mm/week)": irri["irrigation_mm_week"],
        "Category": irri["category"],
        "Breakdown": {
            "Base crop need (mm/wk)": irri["base_mm_week"],
            "Soil factor": irri["soil_factor"],
            "Temp factor": irri["temp_factor"],
            "Humidity factor": irri["humidity_factor"],
            "Effective rainfall (mm/wk)": irri["effective_rain_mm"],
        },
        "Notes": irri["notes"],
    }


In [9]:
# Example runs (you can edit values and re-run) 

# Example 1: Typical kharif scenario
out1 = irrigation_pipeline(
    soil_type="Alluvial soils",
    N=90, P=42, K=43,
    temperature=32, humidity=65,
    ph=6.5, rainfall_mm_week=30  # weekly rainfall in mm
)
out1


{'Predicted Crop': 'muskmelon',
 'Soil Type': 'Alluvial soils',
 'Irrigation (mm/week)': 11.24,
 'Category': 'Low',
 'Breakdown': {'Base crop need (mm/wk)': 35,
  'Soil factor': 1.0,
  'Temp factor': 1.06,
  'Humidity factor': 0.95,
  'Effective rainfall (mm/wk)': 24.0},
 'Notes': []}

In [10]:
# Another example on Laterite soils (porous) with low rainfall 

out2 = irrigation_pipeline(
    soil_type="Laterite soils",
    N=60, P=55, K=44,
    temperature=34, humidity=55,
    ph=6.8, rainfall_mm_week=5
)
out2

{'Predicted Crop': 'muskmelon',
 'Soil Type': 'Laterite soils',
 'Irrigation (mm/week)': 41.08,
 'Category': 'High',
 'Breakdown': {'Base crop need (mm/wk)': 35,
  'Soil factor': 1.15,
  'Temp factor': 1.12,
  'Humidity factor': 1.0,
  'Effective rainfall (mm/wk)': 4.0},
 'Notes': ['Porous soil – consider more frequent, lighter irrigations.']}

In [11]:
# Batch mode (optional) - run multiple fields/records at once ---

batch_inputs = pd.DataFrame([
    # soil_type, N, P, K, temperature, humidity, ph, rainfall_mm_week
    ["Black soils", 80, 40, 40, 29, 58, 7.0, 0],
    ["Red and Yellow soils", 50, 30, 20, 36, 45, 6.2, 10],
    ["Alluvial soils", 85, 58, 41, 31, 70, 7.1, 50],
], columns=["soil_type","N","P","K","temperature","humidity","ph","rainfall_mm_week"])

batch_results = []
for _, r in batch_inputs.iterrows():
    res = irrigation_pipeline(
        soil_type=r["soil_type"],
        N=r["N"], P=r["P"], K=r["K"],
        temperature=r["temperature"],
        humidity=r["humidity"],
        ph=r["ph"],
        rainfall_mm_week=r["rainfall_mm_week"]
    )
    batch_results.append(res)

pd.DataFrame(batch_results)

,Predicted Crop,Soil Type,Irrigation (mm/week),Category,Breakdown,Notes
0,muskmelon,Black soils,29.75,Medium,"{'Base crop need (mm/wk)': 35, 'Soil factor': ...",[Cracking clay – use deeper but less frequent ...
1,muskmelon,Red and Yellow soils,37.43,Medium,"{'Base crop need (mm/wk)': 35, 'Soil factor': ...",[High evapotranspiration risk (>35°C). Monitor...
2,jute,Alluvial soils,6.35,Low,"{'Base crop need (mm/wk)': 50, 'Soil factor': ...",[]
